In [1]:
import pandas as pd
from numpy import random as npr
from modules import utils

In [2]:
SEED = 2021
npr.seed(SEED)

In [3]:
%%time
df = pd.read_csv("data/rating.csv")
utils.evaluate_autorec(df)

DeepCTR-PyTorch version 0.2.7 detected. Your version is 0.2.5.
Use `pip install -U deepctr-torch` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR-Torch/releases/tag/v0.2.7


Load data finished. Number of users: 6040 Number of items: 3706
IAutoRec.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2021-08-10 06:43:47.589506: I tensorflow/core/platform/cpu_feature_guard.cc:143] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2021-08-10 06:43:47.597549: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2494140000 Hz
2021-08-10 06:43:47.597724: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7efde4000b60 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-10 06:43:47.597747: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-08-10 06:43:47.600372: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-08-10 06:43:47.600400: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: UNKNOWN ERROR (303)
2021-08-10 06:43:47.600424: I tenso

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Network built
Epoch: 0000; RMSE:0.24631128239448502; MAE:0.19352556505105042
Epoch: 0003; RMSE:0.20662718558394014; MAE:0.16865199640295644
Epoch: 0006; RMSE:0.19050209714656205; MAE:0.15179304420360515
Epoch: 0009; RMSE:0.1868493569108845; MAE:0.1470646763420728
Epoch: 0012; RMSE:0.18484266517596268; MAE:0.14609042376509876
Epoch: 0015; RMSE:0.18293966338374243; MAE:0.14445570756803608
Epoch: 0018; RMSE:0.18145178284874366; MAE:0.14315700522725164
Epoch: 0021; RMSE:0.18001971618466786; MAE:0.14178786257395679
Epoch: 0024; RMSE:0.1786287865494927; MAE:0.14091151587754555
Epoch: 0027; RMSE:0.17731054476684355; MAE:0.13941548399363174
Epoch: 0030; RMSE:0.1762148542908691; MAE:0.13868728236734182
Epoch: 0033; RMSE:0.17551544730265123; MAE:0.13808812818948848
Epoch: 0036; RMSE:0.1749058642227231; MAE:0.13734221445671446
Epoch: 0039; RMSE:0.17420136688751844; MAE:0.13691623342297887
Epoch: 0042; RMSE:0.17394331968581803; MAE:0.13655306882706436
Epoch: 0045; RMSE:0.173400508433643; MAE:0.135

Epoch: 0393; RMSE:0.18844624346687808; MAE:0.14800365108309094
Epoch: 0396; RMSE:0.187416478767034; MAE:0.14689799819869506
Epoch: 0399; RMSE:0.18647032260321894; MAE:0.14561326205059705
Epoch: 0402; RMSE:0.1876594513267333; MAE:0.14727476841308423
Epoch: 0405; RMSE:0.1869212833528929; MAE:0.14624009931414708
Epoch: 0408; RMSE:0.18753809984766723; MAE:0.1471983668443133
Epoch: 0411; RMSE:0.18855838912524006; MAE:0.1481384664429036
Epoch: 0414; RMSE:0.18827138612423952; MAE:0.14773431049207034
Epoch: 0417; RMSE:0.18782396619812228; MAE:0.14732404207477168
Epoch: 0420; RMSE:0.1871029407474097; MAE:0.14648804863870013
Epoch: 0423; RMSE:0.18844814581639285; MAE:0.14792971456186202
Epoch: 0426; RMSE:0.19002504801227196; MAE:0.14959521697928688
Epoch: 0429; RMSE:0.1867640999400321; MAE:0.14577620630333893
Epoch: 0432; RMSE:0.18731043739914274; MAE:0.14668698454818258
Epoch: 0435; RMSE:0.1873597175885583; MAE:0.14673038876415803
Epoch: 0438; RMSE:0.18680949570536723; MAE:0.14601695656503239
E

0.17181535073330848